In [6]:
#Install and import the required packages.

# !pip install keybert
# !pip install KeyphraseCountVectorizer

import pandas as pd
from keyphrase_vectorizers import KeyphraseCountVectorizer

from keybert import KeyBERT

In [7]:
all_files = ['arsenal.csv',
'aston_vila.csv',
'chelsea.csv',
'everton.csv',
'leeds.csv',
'leicester.csv',
'man_city.csv',
'man_united.csv']

tweet_types = ["positive", "negative", "neutral"]

In [8]:
# This allows us to specify the length of the keywords and make them into keyphrases.
vectorizer = KeyphraseCountVectorizer()

# Print parameters
print(vectorizer.get_params())

kw_model = KeyBERT()

{'binary': False, 'dtype': <class 'numpy.int64'>, 'lowercase': True, 'max_df': None, 'min_df': None, 'pos_pattern': '<J.*>*<N.*>+', 'spacy_pipeline': 'en_core_web_sm', 'stop_words': 'english', 'workers': 1}


In [12]:
for filenames in all_files:
    df = pd.read_csv("..\\data\\" + filenames)
    df = df.drop_duplicates(subset=['processed_tweet'])

    #Use tweets where the extracted sentiment is greater than 55% according to the cardiffnlp sentiment analysis model.
    for tweet_type in tweet_types:
        positive_df = df[df[tweet_type] > 0.55] 

        join_tweets = ""
        all_tweets = []

        for index, row in positive_df.iterrows():
                
            f=len(join_tweets.split()) 
            tweet_length = len(row['processed_tweet'].split())
            #join a set of tweets if the tweet length of them are less than 40 words.
            if f + tweet_length <= 40:
                join_tweets = join_tweets + " " + str(row['processed_tweet'])
            
            else:
                all_tweets.append(join_tweets)
                join_tweets = str(row['processed_tweet'])

       

        # passing KeyphraseCountVectorizer as the vectorizer helps us be flexible with the n-grams of the keywords that are extracted.
        # We would not need to specify a defined number of words (n-grams) that should be found in the keyword.
        keywords = kw_model.extract_keywords(docs=all_tweets, vectorizer=KeyphraseCountVectorizer())

        tweet_detail = "tweet" + tweet_type

        dict = {tweet_detail : all_tweets, 'keyphrases' : keywords}

        new_df = pd.DataFrame(dict)

        new_df.to_csv(tweet_type + "_" + filenames)
